In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import time
import joblib

ModuleNotFoundError: No module named 'pandas'

In [4]:
# Charger les embeddings des datasets
embeddings_fake_news_df = pd.read_csv("data/Embedded/embeddings_fake_news_base_bert.csv")
embeddings_isot_df = pd.read_csv("data/Embedded/embeddings_isot_base_bert.csv")
embeddings_fake_real_df = pd.read_csv("data/Embedded/embeddings_fake_real_base_bert.csv")

# True de Fake-News et Fake de ISOT

On se propose de tester ce que cela donne lorsque l'on entraine sur un mix des datasets, comme nous l'avons fait pour le TF-IDF

In [7]:
# Création de notre dataset
embeddings_isot_df_sub = embeddings_isot_df.sample(frac=0.5, random_state=42)
true_from_fake_news = embeddings_fake_news_df[embeddings_fake_news_df['label'] == 0]
fake_from_isot = embeddings_isot_df_sub[embeddings_isot_df_sub['label'] == 1]

combined_df = pd.concat([true_from_fake_news, fake_from_isot], ignore_index=True)
print(combined_df['label'].value_counts())

label
1    11784
0    10387
Name: count, dtype: int64


In [10]:
# Réalisation du modèle
X = combined_df.drop(columns=['label']).values
y = combined_df['label'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement
classifier_combined = LogisticRegression(max_iter=1000)
start_time = time.time()
classifier_combined.fit(X_train, y_train)
print(f"Temps pour fit : {time.time() - start_time:.2f} secondes")

# Évaluation
y_pred = classifier_combined.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Temps pour fit : 4.09 secondes
Accuracy: 0.9950394588500564
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2097
           1       0.99      1.00      1.00      2338

    accuracy                           1.00      4435
   macro avg       1.00      0.99      1.00      4435
weighted avg       1.00      1.00      1.00      4435


Confusion Matrix:
 [[2083   14]
 [   8 2330]]


In [11]:
# On enregistre le modèle
joblib.dump(classifier_combined, "Modele/base_bert_logistic_combined.pkl")

['Modele/base_bert_logistic_combined.pkl']

In [13]:
# Test sur fake_real

X_fake_real = embeddings_fake_real_df.drop(columns=['label']).values
y_fake_real = embeddings_fake_real_df['label'].values

# Prédictions
y_fake_real_pred = classifier_combined.predict(X_fake_real)

# Évaluation
print("Accuracy:", accuracy_score(y_fake_real, y_fake_real_pred))
print(classification_report(y_fake_real, y_fake_real_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_fake_real, y_fake_real_pred))

Accuracy: 0.5318074191002368
              precision    recall  f1-score   support

           0       0.53      0.66      0.58      3171
           1       0.54      0.40      0.46      3164

    accuracy                           0.53      6335
   macro avg       0.53      0.53      0.52      6335
weighted avg       0.53      0.53      0.52      6335


Confusion Matrix:
 [[2088 1083]
 [1883 1281]]


Et ceci est étonamment mauvais...

# Mélange des deux datasets

In [15]:
# Création de dataset mélangé

# Séparer les vraies et fausses news
isot_real = embeddings_isot_df_sub[embeddings_isot_df_sub['label'] == 1]
isot_fake = embeddings_isot_df_sub[embeddings_isot_df_sub['label'] == 0]

fake_news_real = embeddings_fake_news_df[embeddings_fake_news_df['label'] == 1]
fake_news_fake = embeddings_fake_news_df[embeddings_fake_news_df['label'] == 0]

# On sépare aléatoirement nos datasets
isot_real_1 = isot_real.sample(frac=0.5, random_state=42)
isot_real_2 = isot_real.drop(isot_real_1.index)

isot_fake_1 = isot_fake.sample(frac=0.5, random_state=42)
isot_fake_2 = isot_fake.drop(isot_fake_1.index)

fake_news_real_1 = fake_news_real.sample(frac=0.5, random_state=42)
fake_news_real_2 = fake_news_real.drop(fake_news_real_1.index)

fake_news_fake_1 = fake_news_fake.sample(frac=0.5, random_state=42)
fake_news_fake_2 = fake_news_fake.drop(fake_news_fake_1.index)

# Créer mixed_data_1 avec 50 % des vraies et fausses informations de chaque dataset
mixed_data_1 = pd.concat([isot_real_1, isot_fake_1, fake_news_real_1, fake_news_fake_1], ignore_index=True)

# Créer mixed_data_2 avec le reste des observations non présentes dans mixed_data_1
mixed_data_2 = pd.concat([isot_real_2, isot_fake_2, fake_news_real_2, fake_news_fake_2], ignore_index=True)

# Mélanger les datasets pour éviter toute structure dans les données
mixed_data_1 = mixed_data_1.sample(frac=1, random_state=5).reset_index(drop=True)
mixed_data_2 = mixed_data_2.sample(frac=1, random_state=6).reset_index(drop=True)

# Vérification
print("Mixed Data 1:")
print(mixed_data_1['label'].value_counts())

print("\nMixed Data 2:")
print(mixed_data_2['label'].value_counts())

Mixed Data 1:
label
1    11079
0    10526
Name: count, dtype: int64

Mixed Data 2:
label
1    11079
0    10526
Name: count, dtype: int64


In [18]:
# Notre modèle sur mixed_data_1

# On réalise le split des données

X = mixed_data_1.drop(columns=['label']).values
y = mixed_data_1['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#On applique une régression logistique
classifier_mixed_1 = LogisticRegression(max_iter=1000)
classifier_mixed_1.fit(X_train, y_train)

# Et on évalue notre petit modèle
y_pred = classifier_mixed_1.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9453830131913908

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.94      2075
           1       0.96      0.94      0.95      2246

    accuracy                           0.95      4321
   macro avg       0.95      0.95      0.95      4321
weighted avg       0.95      0.95      0.95      4321


Confusion Matrix:
 [[1976   99]
 [ 137 2109]]


In [19]:
# Test sur Fake_real
# Prédictions
y_fake_real_pred = classifier_mixed_1.predict(X_fake_real)

# Évaluation
print("Accuracy:", accuracy_score(y_fake_real, y_fake_real_pred))
print(classification_report(y_fake_real, y_fake_real_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_fake_real, y_fake_real_pred))

Accuracy: 0.7226519337016575
              precision    recall  f1-score   support

           0       0.84      0.55      0.66      3171
           1       0.66      0.90      0.76      3164

    accuracy                           0.72      6335
   macro avg       0.75      0.72      0.71      6335
weighted avg       0.75      0.72      0.71      6335


Confusion Matrix:
 [[1741 1430]
 [ 327 2837]]


In [20]:
# Notre modèle sur mixed_data_2

# On réalise le split des données

X = mixed_data_2.drop(columns=['label']).values
y = mixed_data_2['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#On applique une régression logistique
classifier_mixed_2 = LogisticRegression(max_iter=1000)
classifier_mixed_2.fit(X_train, y_train)

# Et on évalue notre petit modèle
y_pred = classifier_mixed_2.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9435315899097431

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.94      2088
           1       0.95      0.94      0.95      2233

    accuracy                           0.94      4321
   macro avg       0.94      0.94      0.94      4321
weighted avg       0.94      0.94      0.94      4321


Confusion Matrix:
 [[1979  109]
 [ 135 2098]]


In [21]:
# Test sur Fake_real
# Prédictions
y_fake_real_pred = classifier_mixed_2.predict(X_fake_real)

# Évaluation
print("Accuracy:", accuracy_score(y_fake_real, y_fake_real_pred))
print(classification_report(y_fake_real, y_fake_real_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_fake_real, y_fake_real_pred))

Accuracy: 0.7291239147592739
              precision    recall  f1-score   support

           0       0.84      0.57      0.68      3171
           1       0.67      0.89      0.77      3164

    accuracy                           0.73      6335
   macro avg       0.76      0.73      0.72      6335
weighted avg       0.76      0.73      0.72      6335


Confusion Matrix:
 [[1800 1371]
 [ 345 2819]]


Conclusion de cette exploration : 
- le mélange entre ISOT et fake_news n'apporte de réel gains que si l'on prends des vraies nouvelles et des fake news
- Evalué sur fake_real, on a gain limité, de 7 points.